In [1]:
!pip install --upgrade pip
!pip install python-decouple
!pip install geoalchemy2
!pip install shapely
!pip install scipy
!pip install category-encoders

Requirement already up-to-date: pip in c:\users\albeh\anaconda3\lib\site-packages (19.1.1)


In [2]:
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker
from decouple import config
from shapely import wkb, wkt
from shapely.geometry import Point
from geoalchemy2.shape import to_shape 

import pandas as pd
import numpy as np
import random
import json
from datetime import datetime, timedelta
import re
from matplotlib import pyplot as plt
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, \
                            mean_squared_error
from xgboost import XGBRegressor

pd.options.display.max_columns = None
from sklearn.preprocessing import OneHotEncoder
from category_encoders import BinaryEncoder

In [12]:
!pip install tpot
from tpot import TPOTRegressor

  Stored in directory: C:\Users\albeh\AppData\Local\pip\Cache\wheels\3c\85\2b\2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
  Stored in directory: C:\Users\albeh\AppData\Local\pip\Cache\wheels\22\ea\bf\dc7c8a2262025a0ab5da9ef02282c198be88902791ca0c6658
Successfully built stopit deap


C:\Users\albeh\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [3]:
"""Contains models for DB."""

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, BigInteger, Integer, String, DateTime, ForeignKey, Float
from sqlalchemy.orm import relationship
from geoalchemy2 import Geometry


BASE = declarative_base()


class City(BASE):
    """City model for DB. Has information of cities."""
    __tablename__ = 'city'
    id            = Column(BigInteger, primary_key=True)
    city          = Column(String, unique=False, nullable=False)
    state         = Column(String, unique=False, nullable=True)
    country       = Column(String, unique=False, nullable=False)
    location      = Column(Geometry(geometry_type='POINT'), nullable=False)
    blocks        = relationship("Blocks", back_populates="city")
    zipcodes      = relationship("ZipcodeGeom", back_populates="city")
    incidents     = relationship("Incident", back_populates="city")


class Blocks(BASE):
    """Block model for DB. Has information of city blocks for a related city
        id."""
    __tablename__ = 'block'
    id            = Column(BigInteger, primary_key=True)
    cityid        = Column(BigInteger, ForeignKey('city.id'), nullable=False)
    shape         = Column(Geometry(geometry_type='MULTIPOLYGON'), nullable=False)
    population    = Column(Integer, nullable=False)
    city          = relationship("City", back_populates="blocks")
    incidents     = relationship("Incident", back_populates="block")

class ZipcodeGeom(BASE):
    """Zipcode geometry model for DB. Has information of zipcodes and related
        city id."""
    __tablename__ = 'zipcodegeom'
    id            = Column(BigInteger, primary_key=True)
    cityid        = Column(BigInteger, ForeignKey('city.id'), nullable=False)
    zipcode       = Column(String, nullable=False, unique=True)
    shape         = Column(Geometry(geometry_type='MULTIPOLYGON'), nullable=False)
    city          = relationship("City", back_populates="zipcodes")

class Incident(BASE):
    """Incident model for DB. Has information of a specific crime, including
        where it took place, when it took place, and the type of crime that
        occurred."""
    __tablename__ = 'incident'
    id            = Column(BigInteger, primary_key=True)
    crimetypeid   = Column(BigInteger, ForeignKey('crimetype.id'), nullable=False)
    locdescid     = Column(BigInteger, ForeignKey('locdesctype.id'), nullable=False)
    cityid        = Column(BigInteger, ForeignKey('city.id'), nullable=False)
    blockid       = Column(BigInteger, ForeignKey('block.id'), nullable=False)
    location      = Column(Geometry(geometry_type='POINT'), nullable=False)
    datetime      = Column(DateTime, nullable=False)
    hour          = Column(Integer, nullable=False)
    dow           = Column(Integer, nullable=False)
    month         = Column(Integer, nullable=False)
    year          = Column(Integer, nullable=False)
    city          = relationship("City", back_populates="incidents")
    block         = relationship("Blocks", back_populates="incidents")
    crimetype     = relationship("CrimeType", back_populates="incidents")
    locationdesc  = relationship("LocationDescriptionType", back_populates="incidents")

class CrimeType(BASE):
    """CrimeType model for DB. Has information of the types of crime, including
        a general description and the numerical severity of the crime."""
    __tablename__ = 'crimetype'
    id            = Column(BigInteger, primary_key=True)
    category      = Column(String, unique=True, nullable=False)
    severity      = Column(Integer, nullable=False)
    incidents     = relationship("Incident", back_populates="crimetype")


class LocationDescriptionType(BASE):
    """Location description model for DB. Has information on the type of
        location that the crime took place."""
    __tablename__ = 'locdesctype'
    id            = Column(BigInteger, primary_key=True)
    key1          = Column(String, nullable=False)
    key2          = Column(String, nullable=False)
    key3          = Column(String, nullable=False)
    incidents     = relationship("Incident", back_populates="locationdesc")

In [5]:
class GetDataFull(object):
    def go(self, SESSION, start_year, end_year):
        SQL_QUERY = \
            f'''
            SELECT incident.blockid,
                    incident.datetime,
                    incident.year, 
                    incident.month, 
                    incident.dow, 
                    incident.hour,
                    SUM(crimetype.severity)/AVG(block.population) AS severity
            FROM incident
            INNER JOIN block ON incident.blockid = block.id INNER JOIN crimetype ON incident.crimetypeid = crimetype.id AND block.population > 0
                AND block.population > 0
                AND severity > 0
                AND incident.cityid = 1
                AND incident.year >= {start_year}
                AND incident.year <= {end_year}
            GROUP BY
                incident.blockid,
                incident.datetime,
                incident.year,
                incident.month,
                incident.dow,
                incident.hour
            '''
        return SESSION.execute(text(SQL_QUERY)).fetchall()

In [6]:
def days_in_month(year, month):
    p = pd.Period(f'{year}-{month}-1')
    return p.days_in_month

def day_of_week(dt):
    return dt.weekday()

def create_arrays(blockids, start_year, end_year):
    idx = 0
    X_blockid, X_datetime, X_year, X_month, X_day, X_dow, X_hour, X_risk = [], [], [], [], [], [], [], []
    for blockid in blockids:
        for year in range(start_year, end_year + 1):
            for month in range(1, 12 + 1):      # month range is 1-12
                for day in range(1, days_in_month(year, month) + 1):
                    for hour in range(24):      # hour range is 0-23
                        X_blockid.append(blockid)
                        X_datetime.append(datetime(year, month, day, hour, 0, 0, 0))
                        X_year.append(year)
                        X_month.append(month)
                        X_day.append(day)
                        X_dow.append(day_of_week(datetime(year, month, day)))
                        X_hour.append(hour)
                        X_risk.append(0.0)
                        idx += 1
    
    X = pd.DataFrame({'blockid':  X_blockid,
                      'datetime': X_datetime,
                      'year':     X_year,
                      'month':    X_month,
                      'day':      X_day,
                      'dow':      X_dow,
                      'hour':     X_hour,
                      'risk':     X_risk})

    return X

In [7]:
def process_data_full(data, start_year, end_year):

    def remove_outliers_from_risk(risk):
        std = np.std(risk)
        risk = np.where(risk < 20*std, 
                     risk, 
                     [0.]*len(risk)).reshape(risk.shape)

        return risk
    
    def binary_encode_blockids(X):
        encoded_blockids = pd.DataFrame(BinaryEncoder(cols=['blockid']) \
                                        .fit_transform(X))
        
        X = pd.concat([X, encoded_blockids], axis=1,
                      names=['blockid' + str(i) for i in range(1, 801 +1)])
        X = X.drop(columns=['blockid'])
        return X
    
    NUM_BLOCKIDS = 801
    
    delta_years = end_year - start_year + 1
    
    X = create_arrays(range(1, NUM_BLOCKIDS + 1), start_year, end_year)
    X = X.drop(columns=['year', 'month', 'day', 'dow', 'hour'], axis=1)

    # records is the list of rows we get from the query with this order:
    #   blockid, datetime, year, month, dow, hour, risk
    #   month is from 1 - 12
    
    X1 = []
    for r in data:
        X1.append((r[0], r[1], r[6]))

    X1 = pd.DataFrame(data=X1,
                      columns=['blockid', 'datetime', 'risk2'])
    X1['risk2'] = remove_outliers_from_risk(X1['risk2'].astype(float))
    
    return X1, X

#     X = pd.merge(X, X1, 
#                  how='left',
#                  left_on=['blockid', 'datetime'],
#                  right_on=['blockid', 'datetime']
#                 )
#     X['all_risk'] = X.risk.astype(float) + X.risk2.astype(float)
#     df = X.drop(columns=['risk', 'risk2']) \
#          .rename(mapper={'all_risk': 'risk'}, axis=1)
    
#     y = df['risk'].copy()
#     df = df.drop(columns=['risk'])
    
#     y = remove_outliers_from_risk(y)
#     df = binary_encode_blockids(df)
#     df['risk'] = y
#     df1 = df.iloc[:, :-2]
#     df2 = df.iloc[:, -1]
#     df = pd.concat([df1, df2], axis=1)
    
#     return df

In [8]:
from contextlib import contextmanager

@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""

    DB_URI  = config('DB_URI')
    ENGINE  = create_engine(DB_URI)
    Session = sessionmaker(bind=ENGINE)
    SESSION = Session()
    
    try:
        yield SESSION
        SESSION.commit()
    except:
        SESSION.rollback()
        raise
    finally:
        SESSION.close()


def ready_data_full(training_start_year, training_end_year,
                    testing_start_year, testing_end_year):
    with session_scope() as session:
        training_data = GetDataFull().go(session,
                                         training_start_year,
                                         training_end_year)
        testing_data = GetDataFull().go(session,
                                         testing_start_year,
                                         testing_end_year)
        train, train_generated = process_data_full(training_data,
                                                 training_start_year, 
                                                 training_end_year)
        test, test_generated = process_data_full(testing_data,
                                               testing_start_year, 
                                               testing_end_year)

    return train, test, train_generated, test_generated

In [9]:
%%time
train, test, train_generated, test_generated = ready_data_full(2015, 2016, 2017, 2018)

Wall time: 2min 30s


In [10]:
train.shape, test.shape, train_generated.shape, test_generated.shape

((507148, 3), (508511, 3), (14052744, 3), (14033520, 3))

In [13]:
train.head()

,blockid,datetime,risk2
0,1,2015-01-01 00:00:00,0.000605
1,1,2015-01-01 02:00:00,0.000908
2,1,2015-01-02 10:30:00,0.000605
3,1,2015-01-03 09:40:00,0.000303
4,1,2015-01-03 22:30:00,0.001816


In [14]:
train2 = train.set_index('datetime')

In [39]:
group = train2.groupby('blockid')
# for k in group.groups.keys():
#     g = group.get_group(k)
g = group.get_group(1)
g['risk2'].reset_index(drop=True)
df2 = pd.merge(train_generated, g.reset_index(), 
                 how='left',
                 left_on=['blockid', 'datetime'],
                 right_on=['blockid', 'datetime']
                )
df2.head()

,blockid,datetime,risk,risk2
0,1,2015-01-01 00:00:00,0.0,0.000605
1,1,2015-01-01 01:00:00,0.0,NaN
2,1,2015-01-01 02:00:00,0.0,0.000908
3,1,2015-01-01 03:00:00,0.0,NaN
4,1,2015-01-01 04:00:00,0.0,NaN


In [40]:
df2.dtypes

blockid              int64
datetime    datetime64[ns]
risk               float64
risk2              float64
dtype: object

In [45]:
df2['datetime']= pd.to_timedelta(df2.datetime).dt.total_seconds().astype(int)
df2.head()

C:\Users\albeh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  """Entry point for launching an IPython kernel.


,blockid,datetime,risk,risk2
0,1,1420070400,0.0,0.000605
1,1,1420074000,0.0,NaN
2,1,1420077600,0.0,0.000908
3,1,1420081200,0.0,NaN
4,1,1420084800,0.0,NaN


In [46]:
df2 = df2.fillna(value=0)
df2['all_risk'] = df2['risk'] + df2['risk2']
df2 = df2.drop(columns=['risk', 'risk2']) \
            .rename(mapper={'all_risk': 'risk'}, axis=1)
df2.head()

,blockid,datetime,risk
0,1,1420070400,0.000605
1,1,1420074000,0.000000
2,1,1420077600,0.000908
3,1,1420081200,0.000000
4,1,1420084800,0.000000


In [47]:
df2.sort_values(by=['datetime', 'blockid'], inplace=True, ascending=[True, True])
df2.head()

,blockid,datetime,risk
0,1,1420070400,0.000605
17544,2,1420070400,0.000000
35088,3,1420070400,0.000000
52632,4,1420070400,0.000000
70176,5,1420070400,0.000000


In [48]:
def split_train_validate(df, target_name, test_fraction=0.25):

    test_size = int(df.shape[0] * test_fraction)
    df_train = df.iloc[:df.shape[0]-test_size,  :]
    df_val   = df.iloc[ df.shape[0]-test_size:, :]
    X_train  = df_train.drop(columns=[target_name])
    y_train  = df_train[target_name]
    X_val    = df_val.drop(columns=[target_name])
    y_val    = df_val[target_name]

    print('X_train.shape:', X_train.shape, 'y_train.shape:', y_train.shape)
    print('X_val.shape:', X_val.shape, 'y_val.shape:', y_val.shape)
    
    return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = split_train_validate(df2, 
                                                     'risk', 
                                                     0.25)

X_train.shape: (10539558, 2) y_train.shape: (10539558,)
X_val.shape: (3513186, 2) y_val.shape: (3513186,)


In [49]:
def baseline(y_train, y_val):
    
    mean_value_train = y_train.mean()
    
    y_pred = [abs(y - mean_value_train) for y in y_val]
    mean_value_ypred = np.average(y_pred)
    mae = np.average([abs(y - mean_value_ypred) for y in y_pred], axis = 0)
    return mae

print('Average of the training y against validation y gives MAE:', baseline(y_train, y_val))
print('Average of the training y against itself gives MAE:', baseline(y_train, y_train))

Average of the training y against validation y gives MAE: 6.012492017674962e-08
Average of the training y against itself gives MAE: 5.438305972121914e-08


In [50]:
print(df2['risk'].max())
print(df2['risk'].mean())
print(df2['risk'].median())

0.0018159806295399517
2.7912827272782475e-08
0.0


In [51]:
%%time

tpot = TPOTRegressor(generations=1, population_size=10, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_val, y_val))

Exception ignored in: <bound method Booster.__del__ of <xgboost.core.Booster object at 0x000001C43063A748>>
Traceback (most recent call last):
  File "C:\Users\albeh\Anaconda3\lib\site-packages\xgboost\core.py", line 957, in __del__
    _check_call(_LIB.XGBoosterFree(self.handle))
stopit.utils.TimeoutException


Generation 1 - Current best internal CV score: -1.9946394071306894e-11

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=False, max_features=0.6000000000000001, min_samples_leaf=8, min_samples_split=15, n_estimators=100)
-2.0363419573247403e-11
Wall time: 1h 46min 20s


In [57]:
y_val_predict = tpot.predict(X_val)

In [80]:
from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(y_val, y_val_predict)
RMSE = (np.sqrt(MSE))

print('MSE is {}'.format(MSE))
print('RMSE is {}'.format(RMSE))

MSE is 2.0363419573247403e-11
RMSE is 4.512584577960551e-06


In [52]:
param_grid = {   
    'learning_rate': [0.05],
    'n_estimators':  [50],
    'max_depth': [2],
}
gridsearch = GridSearchCV(XGBRegressor(),
                          param_grid=param_grid, 
                          cv=3, n_jobs=-1,
                          return_train_score=True, verbose=10)

In [59]:
gridsearch.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.6min finished
C:\Users\albeh\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\albeh\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'learning_rate': [0.05], 'n_estimators': [50], 'max_depth': [2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [60]:
gridsearch.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.05, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=50, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [61]:
gridsearch.cv_results_['mean_train_score']

array([-74672773.87127508])

In [62]:
y_pred_gs = gridsearch.predict(X_val)
print('mae:', mean_absolute_error(y_val, y_pred_gs))
print('rmse:', np.sqrt(mean_squared_error(y_val, y_pred_gs)))

mae: 0.03847244355477365
rmse: 0.03847244381942293


In [64]:
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor)
from sklearn.linear_model import LinearRegression

  Stored in directory: C:\Users\albeh\AppData\Local\pip\Cache\wheels\2b\67\89\414471314a2d15de625d184d8be6d38a03ae1e983dbda91e84
Successfully built tabulate


In [65]:
LRmodel = LinearRegression()
LRmodel.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [68]:
y_predLR = LRmodel.predict(X_val)
print('mae:', mean_absolute_error(y_val, y_predLR))
print('rmse:', np.sqrt(mean_squared_error(y_val, y_predLR)))

mae: 7.760622782725684e-08
rmse: 4.512288868211553e-06


In [69]:
pd.Series(LRmodel.coef_,
          X_train.columns).sort_values(ascending=False)

datetime    1.933405e-16
blockid    -2.034532e-10
dtype: float64

In [70]:
RFmodel = RandomForestRegressor(
    n_estimators=100, 
    min_samples_leaf=0.004,
    n_jobs=-1
)

In [71]:
RFmodel.fit(X_train, y_train)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=0.004, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [72]:
y_predRF = RFmodel.predict(X_val)
print('mae:', mean_absolute_error(y_val, y_predRF))
print('rmse:', np.sqrt(mean_squared_error(y_val, y_predRF)))

mae: 5.713708899555593e-08
rmse: 4.512584657087893e-06


In [73]:
pd.Series(RFmodel.feature_importances_, 
          X_train.columns).sort_values(ascending=False)

datetime    0.0
blockid     0.0
dtype: float64

In [75]:
permuter = PermutationImportance(RFmodel, scoring='neg_mean_squared_error', n_iter=1, cv='prefit')
permuter.fit(X_train, y_train)

PermutationImportance(cv='prefit',
           estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=0.004, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
           n_iter=1, random_state=None, refit=True,
           scoring='neg_mean_squared_error')

In [76]:
eli5.show_weights(permuter, top=None, feature_names=X_train.columns.tolist())


C:\Users\albeh\Anaconda3\lib\site-packages\eli5\formatters\html.py:234: RuntimeWarning: invalid value encountered in double_scalars
  rel_weight = (abs(weight) / weight_range) ** 0.7


Weight,Feature
0 ± 0.0000,datetime
0 ± 0.0000,blockid


In [77]:
GBmodel = GradientBoostingRegressor()
GBmodel.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [81]:
y_predGB = GBmodel.predict(X_val)
print('mae:', mean_absolute_error(y_val, y_predGB))
print('rmse:', np.sqrt(mean_squared_error(y_val, y_predGB)))


mae: 5.7258836731097064e-08
rmse: 4.5125845779605515e-06
